# Project: Dermatologist-AI

In this mini project, you will design an algorithm that can visually diagnose [melanoma](http://www.skincancer.org/skin-cancer-information/melanoma), the deadliest form of skin cancer.  In particular, your algorithm will distinguish this malignant skin tumor from two types of benign lesions ([nevi](http://missinglink.ucsf.edu/lm/dermatologyglossary/nevus.html) and [seborrheic keratoses](https://www.aad.org/public/diseases/bumps-and-growths/seborrheic-keratoses)). 

The data and objective are pulled from the [2017 ISIC Challenge on Skin Lesion Analysis Towards Melanoma Detection](https://challenge.kitware.com/#challenge/583f126bcad3a51cc66c8d9a).  As part of the challenge, participants were tasked to design an algorithm to diagnose skin lesion images as one of three different skin diseases (melanoma, nevus, or seborrheic keratosis).  In this project, you will create a model to generate your own predictions.

## Let's prepare the data

In [17]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 10

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(384, 256),  # all images will be resized to 384x256
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = datagen.flow_from_directory(
        'data/valid',  # this is the target directory
        target_size=(384, 256),  # all images will be resized to 384x256
        batch_size=batch_size,
        class_mode='categorical')

test_generator = datagen.flow_from_directory(
        'data/test',  # this is the target directory
        target_size=(384, 256),  # all images will be resized to 384x256
        batch_size=batch_size,
        class_mode='categorical')

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


## Let's load the model

In [19]:
# Load VGG16
from keras import applications
base_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(384, 256, 3))

/anaconda3/envs/dermatologist-ai/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [21]:
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.5))

add_model.add(Dense(3))
add_model.add(Activation('softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 384, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 390, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 192, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 192, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

### And start with the cool stuff

In [26]:
import matplotlib.pyplot as plt

history = model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=150 // batch_size,
        workers=4,
        use_multiprocessing=True)

model.save_weights('first_try.h5')

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()

Epoch 1/100
 26/200 [==>...........................] - ETA: 35:24 - loss: 5.5793 - acc: 0.6538

Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/envs/dermatologist-ai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/envs/dermatologist-ai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/envs/dermatologist-ai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/envs/dermatologist-ai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/e

KeyboardInterrupt: 

In [18]:
model.evaluate_generator(test_generator)

[5.560742884874344, 0.6550000041723252]